In [1]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if ".mp4" in file:
            files.append(os.path.join(r, file))
        
for f in files:
    print(f)
np.array(files).shape

.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_glasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_nightglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_nightnoglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_noglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_sunglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_glasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_nightglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_nightnoglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_noglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_sunglasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_glasses_mix.mp4
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_nightglass

(20,)

In [2]:
for i in range(4):
    print(files[i][57:60])
    print(files[i][61:-4])
    print(files[i][57:-4])
    print('----')

004
004_glasses_mix
004\004_glasses_mix
----
004
004_nightglasses_mix
004\004_nightglasses_mix
----
004
004_nightnoglasses_mix
004\004_nightnoglasses_mix
----
004
004_noglasses_mix
004\004_noglasses_mix
----


In [3]:
from mtcnn.mtcnn import MTCNN
# import the necessary packages
from imutils import face_utils
import dlib
import cv2
import numpy as np
import os
from operator import itemgetter
import time


eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1



def CreateFolder(name):
    try:
        # creating a folder named data 
        if not os.path.exists(name):
            os.makedirs(name)
        # if not created then raise error 
    except OSError: 
        print ('Error: Creating directory of data')            
#==============================================================

detector = MTCNN()

for filename in files:
    
    #==========================
    f = open('Testingdroplist.txt','a')
    f.write(filename+"\n")
    f.close()
    #==========================
    # img = cv2.imread("me.jpg")
    
    #filename = '.\\Dataset\\Training_Evaluation_Dataset\\Training Dataset\\001\\glasses\\yawning.avi'

    print(filename)
    cap = cv2.VideoCapture(filename)
    #================================================================ create new folder

    DefaultPath = '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\'+filename[57:-4]+"\\"
    CreateFolder(DefaultPath)
    CreateFolder(DefaultPath+"face")
    CreateFolder(DefaultPath+"eyes")
    CreateFolder(DefaultPath+"mouth")
    #================================================================
    Face_ImageDataList = []

    currentframe = 0
    missing_frame = 0
    DropList = []

    eyes_w = eyes_width
    eyes_h = eyes_height
    face_w = face_width
    face_h = face_height
    mouth_w = mouth_width
    mouth_h = mouth_height
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(length)
    x,y,w,h = 0,0,0,0
    
    while(cap.isOpened()):
        
        currentframe += 1
        # load the input image and convert it to grayscale
        ret, image = cap.read()
        if (not ret):
          break
        
        #start_time = time.time()
        

        
        #3 frame sample once
        #if(currentframe%3!=0):
        #    continue
        
        #if(currentframe<2200):
        #    continue
        

        height,width = image.shape[:2]
        #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #image = cv2.resize(image,(width//2,height//2),interpolation=cv2.INTER_CUBIC)
        #print(image.shape[:2])
        faces = detector.detect_faces(image)
        #print(np.array(faces).shape[0])
        
        if(np.array(faces).shape[0] > 0):
            face = faces[0]
            x,y,w,h = face['box']
            
            face_image = image[max(0,y):max(0,y+h),max(0,x):max(0,x+w)]
            #print(face_image.shape)
            #print(face_image.shape)
            #print(image[x:x+w,y:y+h])
            #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            face_image = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame" + str(currentframe) + '.jpg',face_image) 


            eyes_image = image[max(0,face['keypoints']['left_eye'][1]-int(h*0.2)):max(0,face['keypoints']['right_eye'][1]+int(h*0.2))
                                ,max(0,face['keypoints']['left_eye'][0]-int(w*0.2)):max(0,face['keypoints']['right_eye'][0]+int(w*0.2))]
            
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame" + str(currentframe) + '.jpg',eyes_image) 


            mouth_image = image[max(0,face['keypoints']['mouth_left'][1]-int(h*0.2)):max(0,face['keypoints']['mouth_right'][1]+int(h*0.2))
                               ,max(0,face['keypoints']['mouth_left'][0]-int(w*0.2)):max(0,face['keypoints']['mouth_right'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame" + str(currentframe) + '.jpg',mouth_image) 



            #cv2.rectangle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)


            #cv2.circle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['right_eye'][0], face['keypoints']['right_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['nose'][0], face['keypoints']['nose'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_left'][0], face['keypoints']['mouth_left'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_right'][0], face['keypoints']['mouth_right'][1]), 2, (0, 255, 0), -1)
            if(currentframe%100==0):        
                name = './data/frame' + str(currentframe) + '.jpg'
                print ('Creating...' + name) 
        elif (missing_frame<60 and (not((x == 0) or (y == 0) or (w == 0) or (h == 0) ))):
            print ('missing using last frame') 
            missing_frame = missing_frame + 1        

            face_image = image[max(0,y):max(0,y+h),max(0,x):max(0,x+w)]
            print(y)
            print(y+h)
            print(x)
            print(x+w)
            #print(image[x:x+w,y:y+h])
            #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            face_image = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame" + str(currentframe) + '.jpg',face_image) 


            eyes_image = image[max(0,face['keypoints']['left_eye'][1]-int(h*0.2)):max(0,face['keypoints']['right_eye'][1]+int(h*0.2))
                                ,max(0,face['keypoints']['left_eye'][0]-int(w*0.2)):max(0,face['keypoints']['right_eye'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame" + str(currentframe) + '.jpg',eyes_image) 


            mouth_image = image[max(0,face['keypoints']['mouth_left'][1]-int(h*0.2)):max(0,face['keypoints']['mouth_right'][1]+int(h*0.2))
                               ,max(0,face['keypoints']['mouth_left'][0]-int(w*0.2)):max(0,face['keypoints']['mouth_right'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame" + str(currentframe) + '.jpg',mouth_image) 
            
            #if(currentframe%100==0):
            name = './data/frame' + str(currentframe) + '.jpg'
            print ('missing using last frame:  '+ 'Creating...' + name) 

        else:
            name = './data/frame' + str(currentframe) + '.jpg'
            f = open('droplist.txt','a')
            f.write(name+"\n")
            f.close()
            DropList.append(currentframe)
            print("drop")



        '''
        cv2.imshow('img',image)

        #print("FPS: ", 1.0 / (time.time() - start_time))

        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
        '''
    
    print("total_frame:" +str(length)+" missing frame"+ str(np.array(DropList).shape[0])+" total frame2:"+ str(currentframe))
    print(DropList)
    cap.release()
    cv2.destroyAllWindows()

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_glasses_mix.mp4
3047
Creating..../data/frame100.jpg
Creating..../data/frame200.jpg


KeyboardInterrupt: 

### Evaluation

In [3]:
import tensorflow as tf

model = tf.keras.models.model_from_json(open('./ModelFace.json').read())
model.load_weights('./ModelFace.h5')

In [4]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 50, 1)    0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 31, 24, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 31, 24, 32)   96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 31, 24, 32)   0           conv1conv2dbn[0][0]              
__________________________________________________________________________________________________
conv2conv2

In [4]:
'''
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Testing Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        
        #print(directory)
        path = str(os.path.join(r, file))
        
        if('eyes' in path):
            AllImageFolder.append(os.path.join(r, file))
        
#for f in AllImageFolder:
#    print(f)
np.array(AllImageFolder).shape
'''

"\nimport os\nimport numpy as np\n\n#path = '.\\Dataset\\Training_Evaluation_Dataset\\Training Dataset\\'\npath = '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image'\n\nAllImageFolder = []\n# r=root, d=directories, f = files\nfor r, d, f in os.walk(path):\n    for file in f:\n        \n        #print(directory)\n        path = str(os.path.join(r, file))\n        \n        if('eyes' in path):\n            AllImageFolder.append(os.path.join(r, file))\n        \n#for f in AllImageFolder:\n#    print(f)\nnp.array(AllImageFolder).shape\n"

In [6]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Testing Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("mix" in directory):
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
    
#llImageFolder = AllImageFolder[2:]
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightnoglasses_mix
.\Dataset\Training

(20,)

In [7]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset'

AllLabelFile = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "head.txt" in file :
            AllLabelFile.append(os.path.join(r, file))
        
for f in AllLabelFile:
    print(f)
    
#AllLabelFile = AllLabelFile [2:]
np.array(AllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_glasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_nightglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_night_noglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_noglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_sunglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_glasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_nightglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_night_noglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_noglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_sunglasses_mixing_head.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_glasses_m

(20,)

In [8]:
num_samples = 167208
batch_size = 250

In [10]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping




def LoadImageandLabel(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:
            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1

            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\face"
            

            #=====find all files in path=======    
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if ".jpg" in file:
                        temp = os.path.join(r, file)
                        Facefiles.append(temp)
                        
                        #Eyefiles.append(temp.replace("face", "eyes"))
                        #Mouthfiles.append(temp.replace("face", "mouth"))


            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            #Eyefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            #Mouthfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            
            #Mouthfiles=Mouthfiles[:-2]
            #Eyefiles=Eyefiles[:-2]
            Facefiles=Facefiles[:-2]
            #print(len(Mouthfiles))
            #print(len(TempLabelList))
            
            for FileIndex in range(len(Facefiles)):
                #======================load label

                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 
                
                #print(framenumber)

                #if(judgeflag == 0 and TempLabelList[framenumber-1] == 1):
                #    judgeflag = 1
                    #print(framenumber-1)

                #Labelarray = np.array(LabelList)
                #print(len(TempLabelList))
                #print(framenumber-1)
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=3)
                LabelList.append(Labelarray)
                #print(LabelList)

                    #Labelarray = np.array(LabelList)
                    #Labelarray = to_categorical(Labelarray, num_classes=2)

                #================================
                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                #ImgRGB = cv2.merge((ImgGray,ImgGray,ImgGray))# this line make gray image to RGB
                #TEMP_ImageDataList.append(ImgRGB)  
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                #print(ImgGray.shape)
                Face_ImageDataList.append(ImgGray)
                #print(np.array(Face_ImageDataList).shape)
                '''
                ImgGray = cv2.imread(Mouthfiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Mouth_ImageDataList.append(ImgGray)


                ImgGray = cv2.imread(Eyefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Eyes_ImageDataList.append(ImgGray)
                '''
                 #====================================
               
                count = count +1
                if count>=batch_size:

                        yield ([np.array(Face_ImageDataList)],np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        LabelList = []

                        
#                   
result= model.evaluate_generator(LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile), steps=ceil(num_samples / batch_size),workers=5,verbose = 1)

669/669 [==============================] - 61s 92ms/step


In [11]:
result

[0.2846651803877789, 0.9330338472936323]